In [24]:
import os
import numpy as np
import tensorflow as tf
from numpy import genfromtxt
from tensorflow.python.util import compat

In [25]:
data = genfromtxt('/root/bandit/bandit_data.csv',
                  delimiter=',', skip_header = 1, dtype = np.float32)

In [26]:
labels = data[:,5]
num_labels = len(set(labels))
labels = (np.arange(num_labels) == np.array(labels)[:,None]).astype(np.float32)

In [27]:
features = data[:,1:3]
reward = data[:,3]
probability = data[:,4]
feature_size = 2

In [28]:
graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.constant(features)
    tf_train_labels = tf.constant(labels)

    weights = tf.Variable(tf.truncated_normal([feature_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))

    train_prediction = tf.nn.softmax(logits)

    arm_choice = tf.argmax(train_prediction, 1)
    input_label = tf.argmax(tf_train_labels, 1)

    arm_equals_input = tf.cast(tf.equal(arm_choice, input_label), tf.float32)
    inverse_prop_score = tf.constant(-reward/probability, dtype = tf.float32)

    ips_output = tf.multiply(arm_equals_input, inverse_prop_score)

    cost = tf.reduce_mean(ips_output)

    optimizer = tf.contrib.opt.ScipyOptimizerInterface(cost, options={'maxiter': 1000000})

In [38]:
model_version = 3
path = os.path.join("/root/bandit/bad_bandit", str(model_version))
builder = tf.saved_model.builder.SavedModelBuilder(path)

In [39]:
tensor_info_x = utils.build_tensor_info(tf_train_dataset)
tensor_info_y = utils.build_tensor_info(train_prediction)

In [40]:
prediction_signature = signature_def_utils.build_signature_def(
      inputs={'features': tensor_info_x},
      outputs={'scores': tensor_info_y},
      method_name=signature_constants.PREDICT_METHOD_NAME)

In [34]:
# legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

In [35]:
# builder.add_meta_graph_and_variables(
#       sess, [tag_constants.SERVING],
#       signature_def_map={
#           'predict_images':
#               prediction_signature
#       },
#       legacy_init_op=legacy_init_op)

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.


In [41]:
with tf.Session(graph = graph) as sess:
    tf.global_variables_initializer().run()
    optimizer.minimize(sess)

    print sess.run([cost, train_prediction])
    
    legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
    
    builder.add_meta_graph_and_variables(
      sess, [tag_constants.SERVING],
      signature_def_map={
          'predict_arm':
              prediction_signature
      },
      legacy_init_op=legacy_init_op)
    builder.save()

INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  Objective function value: -0.370500
  Number of iterations: 0
  Number of functions evaluations: 1
[-0.3705, array([[ 0.42081016,  0.36642113,  0.21276876],
       [ 0.22417471,  0.55909765,  0.21672761],
       [ 0.22417471,  0.55909765,  0.21672761],
       ..., 
       [ 0.42081016,  0.36642113,  0.21276876],
       [ 0.33333334,  0.33333334,  0.33333334],
       [ 0.17312115,  0.49585745,  0.3310214 ]], dtype=float32)]
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /root/bandit/bad_bandit/3/saved_model.pb


In [44]:
!